# TP3 Premiers pas avec MPI
## M1 informatique, Université d'Orléans 2021/2022

L'objectif de ce TP est de mettre en place des programmes simples utilisant les routines MPI pour programmer explicitement le parallélisme.
Dans cette fiche, vous allez utiliser exclusivement les routines de communications point-à-point.

### 1. Premier programme : Hello World

Après avoir installé **OpemMPI** compilez le programme `hello.cpp` ci-dessous à l'aide d'un `Makefile` que vous pouvez personnaliser comme vous le souhaitez.

Pour l'installation vous pouvez vous référer aux instructions données en cours.

~~~make
CXX=mpic++
CXXFLAGS=-std=c++11 -Wall -Wextra -pedantic 
~~~

~~~C++
#include <iostream>
#include <mpi.h>

using namespace std;

int main(int argc, char *argv[])
{
  int pid, nprocs;
  MPI_Init(&argc, &argv);
  MPI_Comm_rank(MPI_COMM_WORLD, &pid);
  MPI_Comm_size(MPI_COMM_WORLD, &nprocs);
  cout << "Bonjour ! Je suis le processus " 
       << pid << " sur " << nprocs
       << " processus." << endl;
  MPI_Finalize();
  return 0;
}
~~~


### 2. Échanges simples entre processeurs

Le programme `echange.cpp` ci-dessous permet au processus d'identifiant 0 d'envoyer un entier au processus d'identifiant 1.

~~~C++
#include <iostream>
#include <mpi.h>

const int tag = 10;

using namespace std;

int main(int argc, char **argv)
{
  int pid, nprocs;
  MPI_Init(&argc, &argv);
  MPI_Comm_rank(MPI_COMM_WORLD, &pid);
  MPI_Comm_size(MPI_COMM_WORLD, &nprocs);

  int a = pid;

  if (pid == 0)
    MPI_Send(&a, 1, MPI_INT, 1, tag, MPI_COMM_WORLD);
  if (pid == 1)
    MPI_Recv(&a, 1, MPI_INT, 0, tag, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

  cout << "je suis " << pid << " et a=" << a << endl;

  MPI_Finalize();
  return 0;
}
~~~

### 3. Maximum d'un tableau

On souhaite paralléliser le calcul du maximum d'un tableau avec les hypothèses suivantes :

1. Le tableau est généré initialement sur le processus d'identifiant 0.
2. Le tableau est de taille *n*, *n* donné en ligne de commande et *n* divisible par le nombre de processus de l'exécution parallèle. 
3. Le résultat final doit être disponible sur le processus d'identifiant 0.

Les étapes à réaliser sont dans l'ordre, les suivantes :

1. Le processus d'identifiant 0 distribue le tableau découpé en parts égales entre tous les processus ;
2. Chaque processus calcule le maximum du morceau de tableau qu'il a reçu ;
3.  Chaque processus envoie son maximum au processus 0 ;
4. Le processus 0 reçoit tous les maximums locaux et finalise le calcul.

### 4. Normalisation d'un vecteur

Soit *V* un vecteur de taille *n* généré sur un processus *root*. On souhaite calculer la normalisation de ce vecteur en partageant le travail sur les différents processus et en rassemblant le résultat sur le processus *root*.

$$
V_{\operatorname{norm}} = \frac{1}{\left\|V\right\|} V \quad\mbox{avec}\quad
\left\|V\right\| = \sqrt{\sum_{i=1}^n V_i^2}
$$

On ne fait plus d'hypothèse sur la taille du vecteur, *n* peut ne pas être divisible par le nombre de processus de l'exécution parallèle.
